In [13]:
import numpy as np
import numpy.linalg as la
import random

def is_transition_matrix(matrix):
    # Check if the matrix is a square matrix
    if len(matrix) != len(matrix[0]):
        return False
    
    # Check if all the values are non-negative
    for row in matrix:
        for value in row:
            if value < 0:
                return False
    
    # Check if each row sums up to 1
    for row in matrix:
        row_sum = sum(row)
        if abs(row_sum - 1) > 1e-9:  # use a tolerance of 1e-9 to account for floating point errors
            return False
    
    # If all the above conditions are satisfied, the matrix is a transition probability matrix
    return True

P = np.array([[0.2, 0.6, 0.2],
              [0.4, 0.1, 0.5],
              [0.1, 0.2, 0.7]])

if is_transition_matrix(P):
    print("The matrix is a transition probability matrix")
else:
    print("The matrix is not a transition probability matrix")








The matrix is a transition probability matrix


In [14]:
# solve for steady state probabilities
eigvals, eigvecs = la.eig (P.T)
# normalize the eigenvector v so that elements sum upto 1. this will give you steady state probability vector.
eigvec = eigvecs[:, np.isclose(eigvals, 1)]
pi = (eigvec / eigvec.sum()).real.T
print("Steady state probabilities:", pi)

Steady state probabilities: [[0.1954023  0.25287356 0.55172414]]


In [15]:
#Simulate Markov Chain
def simulate_markov_chain(P, n, i):
    state = i
    for t in range(n):
        print("Step {}: State {}".format(t, state))
        state = random.choices(range(len(P)), weights=P[state])[0]

# simulate Markov Chain for 10 steps starting from state 0
simulate_markov_chain(P, 12, 0)

Step 0: State 0
Step 1: State 1
Step 2: State 2
Step 3: State 0
Step 4: State 1
Step 5: State 1
Step 6: State 0
Step 7: State 1
Step 8: State 2
Step 9: State 2
Step 10: State 0
Step 11: State 1


In [16]:
#Simulate markov chain multiple
def simulate_markov_chain_multiple(P, n, i, N):
    counts = np.zeros(len(P))          #creates zeros with a length equal to the number of states in the Markov chain
    state = i
    for run in range(N):
        for t in range(n):
            counts[state] += 1
            state = random.choices(range(len(P)), weights=P[state])[0]
    return counts / (n * N)

# simulate Markov Chain for 10 steps starting from state 0 for 10000 runs
proportions = simulate_markov_chain_multiple(P, 12, 0, 10000)
print("Proportions of time spent in each state:", proportions)

Proportions of time spent in each state: [0.19515  0.253175 0.551675]


In [17]:

#Get communication classes
def get_communication_classes(P):
    n = len(P)
    classes = []
    visited = set()
    for i in range(n):
        if i not in visited:
            # find all states that communicate with i
            comm_class = {i}
            queue = [i]
            while queue:
                j = queue.pop(0)
                visited.add(j)
                for k in range(n):
                    if k not in comm_class and P[j, k] > 0:
                        comm_class.add(k)
                        queue.append(k)

In [20]:
# determine if comm_class is recurrent or transient
recurrent_states = np.where(np.isclose(eigvals, 1))[0]
transient_states = np.where(np.abs(eigvals - 1) > 1e-8)[0]

classes = []
for state in recurrent_states:
    class_ = set([state])
    for i in range(P.shape[0]):
        if i not in class_:
            if np.isclose(P[list(class_), i], np.zeros(len(class_))).all() and \
               np.isclose(P[i, list(class_)], np.zeros(len(class_))).all():
                class_.add(i)
    classes.append(class_)

print("Recurrent states:", recurrent_states)
print("Transient states:", transient_states)
print("classes:" , classes)
print("eigenvalue",eigvals)
print(P)

Recurrent states: [2]
Transient states: [0 1]
classes: [{2}]
eigenvalue [-0.36055513  0.36055513  1.        ]
[[0.2 0.6 0.2]
 [0.4 0.1 0.5]
 [0.1 0.2 0.7]]
